### Importing the collection

In [ ]:
import xml.etree.ElementTree as ET
import os
from IPython.display import clear_output
trees = []
path = r'C:\Users\Danyal\Desktop\DCU\Mechanics of Search\CA6005_2022-assignment\CA6005_2022\COLLECTION'
for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    trees.append(ET.parse(fullname))

In [ ]:
trees

In [ ]:
roots = []
for tree in trees:
    roots.append(tree.getroot())

In [ ]:
roots

In [ ]:
#root1=roots[2].findall("TEXT")
for text in roots[1].findall("TEXT"):
    print(text.text)

### Making Three seperate lists for the Text, Headline and the Doc id 

In [ ]:
texts = []
headings = []
doc_id = [] 

for root in roots:
    for text in root.findall("TEXT"):
        texts.append(text.text)
    for head in root.findall("HEADLINE"):
        headings.append(head.text)
    for d_id in root.findall("DOCID"):
        doc_id.append(d_id.text)

### Creating a dataframe with three key values: Text, Headline and the Doc id
#### These three keys will be containing the lists with the data. The length of each list will be the total number of documents

In [ ]:
dataframe = {'document_id':doc_id, 'headlines':headings, 'text':texts}

In [ ]:
dataframe['text'][9]

In [ ]:
#dataframe['text'][10]
def gettext(index):
    return dataframe['text'][index]

def gethead (index):
    return dataframe['headlines'][index]

def getid (index):
    return dataframe['document_id'][index]


In [ ]:
gettext(2)

### Repeating the same steps for the queries

In [ ]:
qtrees = []
path = r'C:\Users\Danyal\Desktop\DCU\Mechanics of Search\CA6005_2022-assignment\CA6005_2022\topics\topics'
for topic in os.listdir(path):
    if not topic.endswith('.xml'): continue
    qfullname = os.path.join(path, topic)
    qtrees.append(ET.parse(qfullname))

In [ ]:
qroots = []
for qtree in qtrees:
    qroots.append(qtree.getroot())

In [ ]:
q_id = []
title = []
desc = [] 
narr = [] 

for qroot in qroots:
    for query_id in qroot.findall("QUERYID"):
        q_id.append(query_id.text)
    for t in qroot.findall("TITLE"):
        title.append(t.text)
    for d in qroot.findall("DESC"):
        desc.append(d.text)
    for n in qroot.findall("NARR"):
        narr.append(n.text)   

In [ ]:
q_dataframe = {'query_id':q_id, 'title':title, 'description':desc, 'narration':narr}

In [ ]:
q_dataframe['description'][5]

In [ ]:
def getq_id(index):
    return q_dataframe['query_id'][index]

def gettitle (index):
    return q_dataframe['title'][index]

def getdesc (index):
    return q_dataframe['description'][index]

def getnarr (index):
    return q_dataframe['narration'][index]

In [ ]:
getdesc(5)

### Creating a new dictionary for further processing (If something wrong happens we can always have a backup)

In [ ]:
data = dataframe

## Pre-Processing of the document text

### Getting rid of the none-type and converting the text to lower case

In [ ]:
def none_type_clean(s):
    if s is None:
        return ''
    else:
        return s

In [ ]:
p = 0
while (p<len(data['headlines'])):
    data['headlines'][p] = none_type_clean(data['headlines'][p]).lower() # = data['headlines'][p].lower()
    p=p+1
p = 0   
while (p<len(data['text'])):
    data['text'][p] = none_type_clean(data['text'][p]).lower() # = data['headlines'][p].lower()
    p=p+1
    
# len(data['headlines'])

In [ ]:
data['text'][2]

### Getting rid of contractions

In [ ]:
import re

# All the english contractions

contractions_dict = { "ain \' t": "are not"," \' s":" is","aren \' t": "are not","can \' t": "can not","can \' t \' ve": "cannot have",
" \' cause": "because","could \' ve": "could have","couldn \' t": "could not","couldn \' t \' ve": "could not have",
"didn \' t": "did not","doesn \' t": "does not","don \' t": "do not","hadn \' t": "had not","hadn \' t \' ve": "had not have",
"hasn \' t": "has not","haven \' t": "have not","he \' d": "he would","he \' d \' ve": "he would have","he \' ll": "he will",
"he \' ll \' ve": "he will have","how \' d": "how did","how \' d \' y": "how do you","how \' ll": "how will","i \' d": "i would",
"i \' d \' ve": "i would have","i \' ll": "i will","i \' ll \' ve": "i will have","i \' m": "i am","i \' ve": "i have",
"isn \' t": "is not","it \' d": "it would","it \' d \' ve": "it would have","it \' ll": "it will","it \' ll \' ve": "it will have",
"let \' s": "let us","ma \' am": "madam","mayn \' t": "may not","might \' ve": "might have","mightn \' t": "might not",
"mightn \' t \' ve": "might not have","must \' ve": "must have","mustn \' t": "must not","mustn \' t \' ve": "must not have",
"needn \' t": "need not","needn \' t \' ve": "need not have","o \' clock": "of the clock","oughtn \' t": "ought not",
"oughtn \' t \' ve": "ought not have","shan \' t": "shall not","sha \' n \' t": "shall not",
"shan \' t \' ve": "shall not have","she \' d": "she would","she \' d \' ve": "she would have","she \' ll": "she will",
"she \' ll \' ve": "she will have","should \' ve": "should have","shouldn \' t": "should not",
"shouldn \' t \' ve": "should not have","so \' ve": "so have","that \' d": "that would","that \' d \' ve": "that would have",
"there \' d": "there would","there \' d \' ve": "there would have",
"they \' d": "they would","they \' d \' ve": "they would have","they \' ll": "they will","they \' ll \' ve": "they will have",
"they \' re": "they are","they \' ve": "they have","to \' ve": "to have","wasn \' t": "was not","we \' d": "we would",
"we \' d \' ve": "we would have","we \' ll": "we will","we \' ll \' ve": "we will have","we \' re": "we are","we \' ve": "we have",
"weren \' t": "were not","what \' ll": "what will","what \' ll \' ve": "what will have","what \' re": "what are",
"what \' ve": "what have","when \' ve": "when have","where \' d": "where did",
"where \' ve": "where have","who \' ll": "who will","who \' ll \' ve": "who will have","who \' ve": "who have",
"why \' ve": "why have","will \' ve": "will have","won \' t": "will not","won \' t \' ve": "will not have",
"would \' ve": "would have","wouldn \' t": "would not","wouldn \' t \' ve": "would not have","y \' all": "you all",
"y \' all \' d": "you all would","y \' all \' d \' ve": "you all would have","y \' all \' re": "you all are","y \' all \' ve": "you all have",
"you \' d": "you would","you \' d \' ve": "you would have","you \' ll": "you will","you \' ll \' ve": "you will have",
"you \' re": "you are","you \' ve": "you have"}

contractions_re=re.compile('(%s)' % "|".join(contractions_dict.keys()))

In [ ]:
# # to add space 
# def add_space(s):            
#     pat = re.compile(r"(['])")
#     return pat.sub(" \\1 ", s)

In [ ]:
# for key in contractions_dict:
#     key1 = add_space(key)
#     contractions_dict[key1] = contractions_dict.pop(key)
#     print(contractions_dict)


In [ ]:
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

In [ ]:
expand_contractions(data['text'][2])

In [ ]:
#data['headlines']=training_corpus['cleaned'].apply(lambda x:expand_contractions(x))
p = 0
while (p<len(data['headlines'])):
    data['headlines'][p] = expand_contractions(data['headlines'][p]) 
    p=p+1

p = 0
while (p<len(data['text'])):
    data['text'][p] = expand_contractions(data['text'][p]) 
    p=p+1

In [ ]:
data['text'][2]

### Cleaning Text

##### Removing words with digits, replace newline characters with whitespace, remove URLs, etc

In [ ]:
# # fUNCTION TO CLEAN

# def clean_text(text):
#     text=re.sub('\w*\d\w*','', text)
#     text=re.sub('\n',' ',text)
#     text=re.sub(r"http\S+", "", text)
#     text=re.sub('[^a-z]',' ',text)
#     return text

In [ ]:
# p = 0
# while (p<len(data['headlines'])):
#     data['headlines'][p] = clean_text(data['headlines'][p]) 
#     p=p+1

# p = 0
# while (p<len(data['text'])):
#     data['text'][p] = clean_text(data['text'][p]) 
#     p=p+1

### Removing punctuation

In [ ]:
import string

def remove_punct(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [ ]:
p = 0
while (p<len(data['headlines'])):
    data['headlines'][p] = remove_punct(data['headlines'][p]) 
    p=p+1

p = 0
while (p<len(data['text'])):
    data['text'][p] = remove_punct(data['text'][p]) 
    p=p+1

### Tokens and stop word removal

In [ ]:
pip install nltk

In [ ]:
# data['text']

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download()
from nltk.stem import PorterStemmer
porter = PorterStemmer()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet

### POS taging and lemmatize

In [ ]:
# Map POS tag to first character lemmatize() accepts
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def lem(text):
    return [wordnet_lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
# lem(data['text'][2])

### Checking if it is working properly

In [ ]:
test = "my name is danyal"
print(lem(test))

In [ ]:
p = 0
while (p<len(data['headlines'])):
    data['headlines'][p] = lem(data['headlines'][p]) 
    p=p+1

p = 0
while (p<len(data['text'])):
    data['text'][p] = lem(data['text'][p]) 
    p=p+1

### Storing the processed text in new variable as a chckpoint

In [ ]:
data1 = data

In [ ]:
# import csv
# a_file = open("data.csv", "w")
# # a_dict = {"a": 1, "b": 2}

# writer = csv.writer(a_file)
# for key, value in data.items():
#     writer.writerow([key, value])

# a_file.close()

In [ ]:
data1['text']

### Creating tokens

In [ ]:
# p = 0
# while (p<len(data['headlines'])):
#     data['headlines'][p] = nltk.word_tokenize(data['headlines'][p])
#     p=p+1

# p = 0
# while (p<len(data['text'])):
#     data['text'][p] = nltk.word_tokenize(data['text'][p])
#     p=p+1
    
# data['text']

### Removing stopwords

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
# p = 0
# while (p<len(data['headlines'])):
#     data['headlines'][p] = [word for word in data['headlines'][p] if word not in stopwords] 
#     p=p+1

# p = 0
# while (p<len(data['text'])):
#     data['text'][p] = [word for word in data['text'][p] if word not in stopwords]
#     p=p+1

In [ ]:
# data['text'][1]

In [ ]:
# def stemmer(text):
#     stemmed = [porter.stem(word) for word in text]
#     return stemmed

In [ ]:
# stemmer(data['text'][2])

In [ ]:
# p = 0
# while (p<len(data['headlines'])):
#     data['headlines'][p] = stemmer(data['headlines'][p])
#     p=p+1

# p = 0
# while (p<len(data['text'])):
#     data['text'][p] = stemmer(data['text'][p])
#     p=p+1

In [ ]:
# p = 0
# q = 0
# while (p<len(data['headlines'])):
#     while(q<len(data['headlines'][p])):
#         data['headlines'][p][q] = stemmer(data['headlines'][p][q])
#         q=q+1
#     p=p+1

# p = 0
# q = 0
# while (p<len(data['text'])):
#     while(q<len(data['text'][p])):
#         data['text'][p][q] = stemmer(data['text'][p][q])
#         q=q+1
#     p=p+1

### Pos Tag

In [ ]:
# p = 0
# while (p<len(data['headlines'])):
#     data['headlines'][p] = nltk.pos_tag(data['headlines'][p]) 
#     p=p+1

# p = 0
# while (p<len(data['text'])):
#     data['text'][p] = nltk.pos_tag(data['text'][p]) 
#     p=p+1

In [ ]:
# data['text']

In [ ]:
# stemmer(data['text'][2][10])

In [ ]:
# p = 0
# q = 0
# while (p<len(data['text'])):
#     while (q<len(data['text'][p])):
#         data['text'][p][q][0] = lem(data['text'][p][q][0])
#         q=q+1
#     p=p+1

In [ ]:
# data['text'][2][1][0]

In [ ]:
# p = 0
# while (p<len(data['headlines'])):
#     data['headlines'][p] = nltk.pos_tag(data['headlines'][p]) 
#     p=p+1

# p = 0
# while (p<len(data['text'])):
#     data['text'][p] = nltk.pos_tag(data['text'][p]) 
#     p=p+1

In [ ]:
# def get_xml_files(path):
#     xml_list = []
#     for filename in os.listdir(path):
#         if filename.endswith(".xml"):
#             xml_list.append(os.path.join(path, filename))
#     return xml_list

# get_xml_files(path)

# from xml.dom import minidom

# documents = []
# for current_doc in get_xml_files(path):
#     documents.append(minidom.parse(current_doc))
# #file = minidom.parse('models.xml')

In [ ]:
# texts = []
# for doc in documents:
#     texts.append(doc.getElementsByTagName('TEXT'))

In [ ]:
# test = "you ' re"
# print(add_space(test))

In [ ]:
# re.compile(r"([.()!])")
# def rem_whitwspace_before(s):
#     for p in range(0, len(s)-1):
#         if s[p] == "'":
# def add_space(s):            
#     pat = re.compile(r"(['])")
#     return pat.sub(" \\1 ", s)

### Installing dill and using it to save the session so that will not have to start from the begining 
#### The lemmatize function took approximately 3 hrs for the text in the corpus

In [ ]:
# pip install dill

In [ ]:
import dill
from itertools import chain

In [ ]:
#dill.dump_session('notebook_env.db')

### Using this command we can restore the session

In [ ]:
dill.load_session('notebook_env.db')

In [ ]:
p = 0
while (p<len(data['headlines'])):
    data['headlines'][p] = [word for word in data['headlines'][p] if word not in stopwords] 
    p=p+1

p = 0
while (p<len(data['text'])):
    data['text'][p] = [word for word in data['text'][p] if word not in stopwords]
    p=p+1

### Combining the headlines and the text

In [ ]:
dataf ={key: value for key, *value in zip(data['document_id'], data['headlines'], data['text'])}

In [ ]:
dataf

In [ ]:
#for i in dataf.keys():
    dataf[i]=list(chain.from_iterable(dataf[i]))

In [ ]:
# dataf

In [ ]:
len(dataf)

In [ ]:
dill.dump_session('notebook_env.db')

### Creating a list containing the Vocabulary

In [ ]:
vocabulary = [items for items in dataf.values()]
vocabulary = list(chain.from_iterable(vocabulary))

In [ ]:
len(vocabulary)

In [ ]:
# Lets get the vocabulary of words we have in each of the doccuments.

total_vocabulary = list(set(vocabulary))
len(total_vocabulary)

## Creating an inverted index:
#### A dictionary containing terms as keys, lists of tuples containing the documents in which that term is present along with the frequency of that term in that document

In [ ]:
inv_index = {}
for x in total_vocabulary:
    for y,z in dataf.items():
        if x in z:
            word_freq = z.count(x)
            inv_index.setdefault(x,[]).append((y,word_freq))

In [ ]:
len(inv_index)

### The inverted index took approximately 5.30 hrs to complete therefore storing it

In [ ]:
%store inv_index
%store dataf
%store total_vocabulary

#### Command for restoring the variables

In [ ]:
%store -r inv_index
%store -r dataf
%store -r total_vocabulary

In [ ]:
inv_index

In [ ]:
inv_index['icontroversy'][0][0]

### Query Processing

In [ ]:
def querry_processing(querry_text):
    
    #Removing None_types from the querries.
    querry_text=none_type_clean(querry_text)
    
    #Changing the case of the querry_text
    querry_text=querry_text.lower()
    
    #Expanding english contractions
    querry_text = expand_contractions(querry_text)
    
    #Removing Punctuations
    querry_text = remove_punct(querry_text)
    
    #Lemmatisation, pos-tagging and tokenisation
    querry_text = lem(querry_text)
    
    #Stop_words removal
    #querry_text = [word for word in querry_text if not word in stopwords]


In [ ]:
querry_text = [word for word in querry_text if not word in stopwords]

In [ ]:
for i in range(0,len(q_dataframe['title'])):
    q_dataframe['title'][i]=querry_processing(q_dataframe['title'][i])
#     q_data['q_desc'][i]=querry_processing(q_data['q_desc'][i])
#     q_data['q_narr'][i]=querry_processing(q_data['q_narr'][i])

In [ ]:
%store -r q_dataframe

In [ ]:
q_data ={key: value for key, value in zip(q_dataframe['query_id'], q_dataframe['title'])}

In [ ]:
%store -r q_data

In [ ]:
# qu = ['oil', 'accident',  'bird']
# def vec_score(q,inv):
#     vec = {}
#     w_score = 0
#     nume = 0
#     doc_score = 0
#     for word in q:
#             for i in range(0, len(inv[word])):
#                 vec[inv[word][i][0]] = (q.count(word) * inv[word][i][1])/(math.sqrt((inv[word][i][1])**2) * (math.sqrt((q.count(word))**2)) )
# #                 w_score = w_score + doc_score
# #             nume = nume+w_score
#     return vec
# len(vec_score(qu, inv_index))

In [ ]:
qu = ['oil', 'accident',  'bird']
# def vec_score(q,inv):
#     vec = {}
#     vec1 = {}
#     doc_list = []
#     for word in q:
#         for i in range (0, len(inv[word])):
#             if inv[word][i][0] in doc_list:
#                 vec[inv[word][i][0]] = vec[inv[word][i][0]] + (q.count(word) * inv[word][i][1])/(math.sqrt((inv[word][i][1])**2) * (math.sqrt((q.count(word))**2)))
#                 vec1[inv[word][i][0]]
#             if inv[word][i][0] not in doc_list:
#                 doc_list.append(inv[word][i][0])
            
#     return vec

# vec_score(qu, inv_index)

### Vector Space Model

In [ ]:
def vec_num (query):
    vecn = {}
    for term in query:
        if term not in inv_index:
            continue
        for i in range (0, len(inv_index[term])):
            if inv_index[term][i][0] not in vecn:
                vecn[inv_index[term][i][0]] = 0
            vecn[inv_index[term][i][0]] += inv_index[term][i][1] * query.count(term)
    return vecn

In [ ]:
def unique(ls): 
    # initialize a null list
    unique_l = []     
    # traverse for all elements
    for x in ls:
        # check if exists in unique_list or not
        if x not in unique_l:
            unique_l.append(x)
    return unique_l

In [ ]:
vecnu = {}
for q_id in q_data:
    vecnu[q_id] = vec_num(q_data[q_id])

In [ ]:
denom1 = {}
t_d = []
for did in dataf:
    denom1[did] = 0
    for term in unique(dataf[did]):
        if term not in t_d:
            t_d.append(term)
            denom1[did] += (dataf[did].count(term))**2
        t_d = []

In [ ]:
import math
#denom1 = {}
for i in denom1.keys():
    denom1[i]=math.sqrt(denom1[i])

In [ ]:
denom1

In [ ]:
denom2 = {}
t_d = []
for qid in q_data:
    denom2[qid] = 0
    for term in q_data[qid]:
        if term not in t_d:
            t_d.append(term)
            denom2[qid] += (q_data[qid].count(term))**2
        t_d = []

In [ ]:
#denom2_ = {}
for i in denom2:
    denom2[i]=math.sqrt(denom2[i])

In [ ]:
denom2

In [ ]:
def denominator(qid):
    denom = {}
    for did in denom1:
        denom[did] = denom1[did] * denom2[qid]
    return denom

In [ ]:
denomf = {}
for qid in q_data:
    denomf[qid] = denominator(qid)

In [ ]:
for qid in vecnu:
    for did in vecnu[qid]:
        vecnu[qid][did] = vecnu[qid][did] / denomf[qid][did]

In [ ]:
cos = {}
cos = vecnu

### Sorting the values

In [ ]:
sorted_vsm = {}
for query in cos.keys():
    sorted_vsm[query] = sorted(cos[query].items(), key=lambda x: x[1], reverse=True)

### Writing the results in a file

In [ ]:
from contextlib import redirect_stdout
with open('outVSM100.txt','w') as file:
    with redirect_stdout(file):
        for query in sorted_vsm:
            count = 1
            for i in range(0, len(sorted_vsm[query])):
                if (count >= 101):
                    break
                print(query,'1',sorted_vsm[query][i][0], count, sorted_vsm[query][i][1],'run1')
                count = count+1

### VSM Using TF-IDF

### Getting TF and IDF

In [ ]:
import numpy as np

idf={}

for word in total_vocabulary:
    idf[word] = np.log2(len(dataf)/len(inv_index[word]))

In [ ]:
idf

### TF-IDF

In [ ]:
tf_idf = {}
for word in total_vocabulary:
    for i in range (0, len(inv_index[word])):
        #tf_idf[word] = idf[word] * inv_index[word][i][1]
        tf_idf.setdefault(word,[]).append((inv_index[word][i][0],idf[word] * inv_index[word][i][1]))
    
tf_idf['regardless']

In [ ]:
%store -r idf
%store -r tf_idf

#### Getting the numerator

In [ ]:
def simi_n(query):
    num_s={}
    for term in query:
        if term not in tf_idf:
            continue
        for i in range(0,len(tf_idf[term])):
            if tf_idf[term][i][0] not in num_s:
                num_s[tf_idf[term][i][0]] = 0
            num_s[tf_idf[term][i][0]]+= tf_idf[term][i][1]*query.count(term)    
    return num_s

In [ ]:
vecnu_sim = {}
for q_id in q_data:
    vecnu_sim[q_id] = simi_n(q_data[q_id])

In [ ]:
import numpy as np

def d1(term):
    d1={}
    for term in tf_idf:
        for i in range(0,tf_idf[term]):
            d1[tf_idf[term][i][0]]=tf_idf[term][i][1]**2
    return d1


d1={}
for did in dataf:
    for term in unique(dataf[did]):
        #for i in range(0,len(unique(data3[did]))):
        if did not in d1:
            d1[did]=0
        d1[did]+=((dataf[did].count(term))*np.log2(len(dataf)/((dataf[did].count(term)))))**2
        #for term in tf_idf:
        
for i in d1.keys():
    d1[i]=math.sqrt(d1[i])

In [ ]:
def denominator1(qid):
    denomina = {}
    for did in d1:
        denomina[did]= d1[did]*denom2[qid]
    return denomina

In [ ]:
denomina_final = {} 
for qid in q_data:
    denomina_final[qid] = denominator1(qid)

In [ ]:
denomina_final

In [ ]:
# Cosine Similarity 

for qid in vecnu_sim:
    for did in vecnu_sim[qid]:
        vecnu_sim[qid][did] = vecnu_sim[qid][did] / denomina_final[qid][did]

In [ ]:
#Storing the cosine similarity in separate dictionary cos1 - for the tfidf approach

cos1 = vecnu_sim

In [ ]:
#Sorting each the smilarity scores

sim_sort1 = {}
for i in cos1.keys():
    sim_sort1[i]=(sorted(cos1[i].items(), key=lambda x: x[1], reverse = True))

In [ ]:
from contextlib import redirect_stdout
with open('outVSMtf10.txt','w') as file:
    with redirect_stdout(file):
        for query in sim_sort1:
            count = 1
            for i in range(0, len(sim_sort1[query])):
                if (count >= 11):
                    break
                print(query,'1',sim_sort1[query][i][0], count, sim_sort1[query][i][1],'run1')
                count = count+1

In [ ]:
# def querry_processing(querry_text):
    
#     #Removing None_types from the querries.
#     querry_text=none_type_clean(querry_text)
    
#     #Changing the case of the querry_text
#     querry_text=querry_text.lower()
    
#     #Expanding english contractions
#     querry_text = expand_contractions(querry_text)
    
#     #Removing Punctuations
#     querry_text = remove_punct(querry_text)
    
#     #Lemmatisation, pos-tagging and tokenisation
#     querry_text = lem(querry_text)
    
#     #Stop_words removal
#     #querry_text = [word for word in querry_text if not word in stopwords]

#     return querry_text

In [ ]:
# querry_text = [word for word in querry_text if not word in stopwords]

In [ ]:
# for i in range(0,len(q_dataframe['title'])):
#     q_dataframe['title'][i]=querry_processing(q_dataframe['title'][i])
# #     q_data['q_desc'][i]=querry_processing(q_data['q_desc'][i])
# #     q_data['q_narr'][i]=querry_processing(q_data['q_narr'][i])

In [ ]:
# %store -r q_dataframe

In [ ]:
# q_dataframe

In [ ]:
# q_data ={key: value for key, value in zip(q_dataframe['query_id'], q_dataframe['title'])}

In [ ]:
# q_data

In [ ]:
# %store -r q_data

In [ ]:
# def q_weight(query):
#     q_freq = {}
#     for i in inv_index.keys():
#         # initially adding all the elements into the dictionary and initialising the values as zero
#         if i not in q_freq:
#             q_freq.update({i: 0})
 
#     for val in query:
#         # to get the number of occurrence of each terms
#         if val in q_freq:
#             q_freq[val] += 1
#             # value incremented by one if the term is found in the documents
 
#     for i in q_freq:
#         q_freq[i] = q_freq[i] / len(query)
#         # term frequency obtained by dividing the number of occurrence of terms by total number of terms in the query
 
#     return q_freq

In [ ]:
# qWeight={}
# for query in q_data:
#     qWeight[query]=q_weight(q_data[query])

In [ ]:
# %store qWeight

In [ ]:
# import math
# def similarity_Computation(query_Weight):
#     ''' Function to calculate the similarity measure in which the weight of the
#     query and the document is multiplied in the numerator and the the weight is
#     squared and squareroot is taken the weights of the query and document'''
 
#     numerator = 0
#     denomi1 = 0
#     denomi2 = 0
#     # initialisation of the variables with zero which is needed for computation
 
#     similarity = {}
#     # initialisation of dictionary which has the name of document as key and the
#     # similarity measure as value
 
#     for terms in inv_index:
#         #for terms in data3[document]:
#             # cosine similarity is calculated
#         for i in range(0,len(tf_idf[terms])):
#                 numerator += tf_idf[terms][i][1] * query_Weight[terms]
#                 denomi1 += tf_idf[terms][i][1] * tf_idf[terms][i][1]
#         denomi2 += query_Weight[terms] * query_Weight[terms]
#             # the summation values of the weight is calculated and later they are
#             # divided
 
#         if denomi1 != 0 and denomi2 != 0:
#             # to avoid the zero division error

#             for i in range(0,len(inv_index[terms])):
#                     simi = numerator / (math.sqrt(denomi1) * math.sqrt(denomi2))

#                     similarity.update({inv_index[terms][i][0]: simi})
#             #dictionary is updated
 
#             numerator = 0
#             denomi2 = 0
#             denomi1 = 0
#             # reinitialisation of the variables to zero
 
#     return (similarity)

In [ ]:
# sim = {}
# for query in qWeight:
#     sim[query] = similarity_Computation(qWeight[query])

In [ ]:
%store sim

In [ ]:
# sim['10.2452/141-AH']

In [ ]:
# sim1 = {}
# for query in sim.keys():
#     sim1[query] = sorted(sim[query].items(), key=lambda x: x[1], reverse=True)

In [ ]:
# sim1['10.2452/141-AH']

In [ ]:
# %store sim1

In [ ]:
# k[0]
# sim

In [ ]:
# from contextlib import redirect_stdout
# with open('out.txt','w') as file:
#     with redirect_stdout(file):
#         for query in sim1:
#             count = 1
#             for i in range(0, len(sim1[query])):
#                 if (count >= 101):
#                     break
#                 print(query,'1',sim1[query][i][0], count, sim1[query][i][1],'run1')
#                 count = count+1

In [ ]:
# from contextlib import redirect_stdout
# with open('out1.txt','w') as file:
#     with redirect_stdout(file):
#         for query in sim1:
#             count = 1
#             for i in range(0, len(sim1[query])):
#                 if (count >= 11):
#                     break
#                 print(query,'1',sim1[query][i][0], count, sim1[query][i][1],'run1')
#                 count = count+1

In [ ]:
# denom = {}
# t_d = []
# for did in dataf:
#     denom[did] = 0
#     for term in dataf[did]:
#         if term not in t_d:
#             t_d.append(term)
#             denom[did] += (dataf[did].count(term))**2
#         t_d = []

In [ ]:
%store denom

In [ ]:
# q_t_d = {}
# for qid in q_data:
#     for i in range(0, len(q_data[qid])):
#         if q_data[qid][i] in inv_index:
#             q_t_d.setdefault(qid, {}).setdefault(q_data[qid][i], []).append(inv_index[q_data[qid][i]])

In [ ]:
# qu = ['oil', 'accident',  'bird']
# num = {}
# for qid in q_data:
#     for query in q_t_d:
#         for i in range(0, len(q_t_d[qid][0])):
#             num[qid] = 
    

## BM25

##### Calculating the average document length

In [ ]:
addition = 0
for doc in dataf:
    addition += len(dataf[doc])
avg_l = addition/len(dataf)

In [ ]:
avg_l

#### BM25 function which can calculate the score of all documents for a given query

In [ ]:
def BM25 (query):
    bm = {}
    for term in query:
        if term not in inv_index:
            continue
        for i in range (0, len(inv_index[term])):
            if inv_index[term][i][0] not in bm:
                bm[inv_index[term][i][0]] = 0
            bm[inv_index[term][i][0]] += (idf[term] * ((inv_index[term][i][1] * 2.5)))/((inv_index[term][i][1] + (2 * ((1 - 0.75) + 0.75 * (len(dataf[inv_index[term][i][0]])/avg_l)))))
    #print(bm)
    return bm

#### Creating a dictionary with query id as keys and value as another dictionary containing document id with their scores

In [ ]:
bm2 = {}
for q_id in q_data:
    bm2[q_id] = BM25(q_data[q_id])

In [ ]:
len(bm2['10.2452/141-AH'])

In [ ]:
# def BM25 (query):
#     for term in query: 
#         for i in range (0, len(inv_index[term])):
#             if inv_index[term][i][0] not in bm:
#                 bm[inv_index[term][i][0]] = 0
#             bm[inv_index[term][i][0]] = bm[inv_index[term][i][0]] / (inv_index[term][i][1] + (2 * (1 - 0.75 + 0.75) * (len(dataf[inv_index[term][i][0]])/avg_l)))
#     #print(bm)
#     return bm

In [ ]:
BM25(q_data['10.2452/141-AH'])

In [ ]:
#inv_index[term][i][1] + (2 * (1 - 0.75 + 0.75) * (len(dataf[inv_index[term][i][0]])/avg_l))

### Sorting the values

In [ ]:
sorted_bm2 = {}
for query in bm2.keys():
    sorted_bm2[query] = sorted(bm2[query].items(), key=lambda x: x[1], reverse=True)

In [ ]:
sorted_bm2_1 = {}
for query in bm2.keys():
    sorted_bm2_1[query] = sorted(bm2[query].items(), key=lambda x: x[1], reverse=True)

In [ ]:
%store sorted_bm2

### Saving the results of BM25 in a file

In [ ]:
from contextlib import redirect_stdout
with open('outbm100.txt','w') as file:
    with redirect_stdout(file):
        for query in sorted_bm2:
            count = 1
            for i in range(0, len(sorted_bm2[query])):
                if (count >= 101):
                    break
                print(query,'1',sorted_bm2[query][i][0], count, sorted_bm2[query][i][1],'run1')
                count = count+1

### Multinomial Language Model

##### Total terms in the corpus

In [ ]:
total = 0
for doc in dataf:
    total += len(dataf[doc])

##### Count of term in the corpus

In [ ]:
total_term = {}
for term in inv_index:
    for i in range(0, len(inv_index[term])):
        if term not in total_term:
            total_term[term] = 0
        total_term[term] += inv_index[term][i][1]

In [ ]:
total_term['first']

#### Function to get the probablity of a given query in documents

In [ ]:
def prob_q(query):
    prob = {}
    for term in query:
        if term not in inv_index.keys():
            continue
        for i in range (0, len(inv_index[term])):
            if inv_index[term][i][0] not in prob:
                prob[inv_index[term][i][0]] = 1
            if term not in dataf[inv_index[term][i][0]]:
                prob[inv_index[term][i][0]] = prob[inv_index[term][i][0]] * (total_term[term] / total)
            prob[inv_index[term][i][0]] *= (inv_index[term][i][1] / len(dataf[inv_index[term][i][0]])) #+ (total_term[term] / total)
    return prob

In [ ]:
prob = {}
for qid in q_data:
    prob[qid] = prob_q(q_data[qid])

In [ ]:
# sorted_prob

##### Sorting the results

In [ ]:
sorted_prob = {}
for query in prob.keys():
    sorted_prob[query] = sorted(prob[query].items(), key=lambda x: x[1], reverse=True)

In [ ]:
sorted_prob

In [ ]:
from contextlib import redirect_stdout
with open('LM100.txt','w') as file:
    with redirect_stdout(file):
        for query in sorted_prob:
            count = 1
            for i in range(0, len(sorted_prob[query])):
                if (count >= 101):
                    break
                print(query,'1',sorted_prob[query][i][0], count, sorted_prob[query][i][1],'run1')
                count = count+1

In [ ]:
%store sorted_prob